# TP2 : Traitement Batch avec Hadoop Streaming

## Prérequis
- Installer Python sur tous les nœuds du cluster.

---

## Partie 2 : Exécution d'un job avec Hadoop Streaming

### Exemple : WordCount

Nous allons coder un exemple de WordCount pour compter le nombre de fois que chaque mot apparaît dans un ensemble de fichiers texte.

### Mapper : mapperWC.py
Créez un fichier `mapperWC.py` avec le code suivant :


In [ ]:

# mapperWC.py
import sys


for line in sys.stdin:
    line = line.strip()
    words = line.split()
    for word in words:
        print(f'{word}	1')


### Reducer : reducerWC.py
Créez un fichier `reducerWC.py` avec le code suivant :


In [ ]:
# reducerWC.py
import sys

current_word = None
current_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split('	', 1)
    try:
        count = int(count)
    except ValueError:
        continue
    if current_word == word:
        current_count += count
    else:
        if current_word:
            print(f'{current_word}	{current_count}')
        current_word = word
        current_count = count

if current_word == word:
    print(f'{current_word}	{current_count}')


---

## Exécution locale

1. Créez un fichier `input.txt` contenant un texte.
2. Exécutez la commande suivante dans un terminal :

```bash
cat input.txt | python mapperWC.py | sort | python reducerWC.py
```

## Exécution sur le cluster Hadoop

1. Transférez les scripts :

```bash
docker cp mapperWC.py namenode:/mapperWC.py
docker cp reducerWC.py namenode:/reducerWC.py
```

2. Accédez au container du namenode et rendez les scripts exécutables :

```bash
docker exec -it namenode bash
chmod u+x mapperWC.py
chmod u+x reducerWC.py
```

3. Créez un répertoire d'entrée dans HDFS et y placer le fichier `input.txt` :

```bash
hdfs dfs -mkdir -p /user/hadoop/input
hdfs dfs -put input.txt /user/hadoop/input
```

4. Exécutez le job Hadoop Streaming :

```bash
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -input /user/hadoop/input \
    -output /user/hadoop/output \
    -mapper /mapperWC.py \
    -reducer /reducerWC.py
```

5. Affichez les résultats :

```bash
hdfs dfs -cat /user/hadoop/output/part-00000
```

6. Nettoyez les fichiers de sortie :

```bash
hdfs dfs -rm -r /user/hadoop/output
```
